<a href="https://colab.research.google.com/github/Du-nara/ME421-Mechanical-Systems-Lab-A3/blob/main/Vibration/E_20_260_Vibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# ============================================================================
# ME421 - VIBRATION MODELING LAB
# ACTIVITY 1: Derive 2-DOF Model for First Two Dominant Vibration Modes
# ============================================================================


# Activity 1: Two-Degree-of-Freedom Model Development

## Objective
Derive a mathematical model with two degrees of freedom (2-DOF) that captures the first two dominant vibration modes of the TecQuipment vibration analyzer system configured as a simply supported beam with central exciter mass.

---

## 1. System Description and Configuration

The TecQuipment TM1016V vibration analyzer consists of:
- A metal beam that can be configured as simply supported (both ends pinned)
- An unbalanced motor (exciter) with eccentric mass mounted at the beam center
- Displacement and acceleration sensors for measurement

### Physical Configuration



    |←---- L/2 = 0.4 m ----→|←---- L/2 = 0.4 m ----→|
    ↓                       ↓                       ↓
[Support]-------------[Exciter Mass]----------[Support]
    O                    m_exciter                  O
    |←------------- L_beam = 0.8 m --------------→|



For a simply supported beam with central mass, two dominant vibration modes exist:
- **Mode 1**: First bending mode (fundamental frequency)
- **Mode 2**: Second bending mode (higher harmonic)

---

## 2. System Parameters

From TM1016V Technical Specifications:


***

## 3. Theoretical Background

### 3.1 Simply Supported Beam Natural Frequencies

For a uniform simply supported beam without added mass, the natural frequencies are given by:

$$\omega_n = n^2 \pi^2 \sqrt{\frac{EI}{m_L L^4}}$$

where:
- $n$ = mode number (1, 2, 3, ...)
- $E$ = Young's modulus
- $I$ = Second moment of area
- $m_L$ = mass per unit length
- $L$ = beam length

### 3.2 Effect of Concentrated Mass (Dunkerley's Method)

When a concentrated mass is added at the center, Dunkerley's method provides an approximation:

$$\frac{1}{\omega_{total}^2} = \frac{1}{\omega_{beam}^2} + \frac{1}{\omega_{mass}^2}$$

The equivalent stiffness at the center for mode $n$ is:

$$k_{eq,n} = \frac{n^4 \pi^4 EI}{L^3}$$

For the first mode (n=1) at center:
$$k_{eq,1} = \frac{48 EI}{L^3}$$

### 3.3 Modal Mass

For the first mode with sinusoidal displacement $y(x) = A\sin(\pi x/L)$:

**Beam modal mass:**
$$m_{beam,modal} = \int_0^L m_L \sin^2\left(\frac{\pi x}{L}\right) dx = \frac{m_{beam}}{2}$$

**Total effective mass (Mode 1):**
$$m_{eff,1} = m_{exciter} + 0.5 \times m_{beam}$$

***

## 4. Mathematical Model - Two Degree of Freedom System

### 4.1 Generalized Coordinates

We use modal coordinates:
- $q_1(t)$ : First mode amplitude
- $q_2(t)$ : Second mode amplitude

### 4.2 Equations of Motion (Undamped)

The equations of motion in matrix form:

$$\mathbf{M}\ddot{\mathbf{q}} + \mathbf{K}\mathbf{q} = \mathbf{F}(t)$$

where:

$$\mathbf{M} = \begin{bmatrix} m_{eff,1} & 0 \\ 0 & m_{eff,2} \end{bmatrix}, \quad
\mathbf{K} = \begin{bmatrix} k_1 & 0 \\ 0 & k_2 \end{bmatrix}, \quad
\mathbf{q} = \begin{bmatrix} q_1 \\ q_2 \end{bmatrix}$$

For decoupled modes (orthogonal mode shapes), the mass and stiffness matrices are diagonal.

***

## 5. Natural Frequency Calculations


In [3]:
# ============================================================================
# 2-DOF MODEL SUMMARY
# ============================================================================

print("\n" + "="*60)
print("TWO-DEGREE-OF-FREEDOM MODEL SUMMARY")
print("="*60 + "\n")

# Modal parameters
modal_params = {
    'Mode': [1, 2],
    'ω_n [rad/s]': [omega_1, omega_2],
    'f_n [Hz]': [f_1, f_2],
    'T_n [s]': [1/f_1, 1/f_2],
    'm_eff [kg]': [m_eff_1, m_eff_2],
    'k_eff [N/m]': [k_equiv_1, omega_2**2 * m_eff_2]
}

df_modal = pd.DataFrame(modal_params)
print(df_modal.to_string(index=False))
print("\n" + "="*60 + "\n")

# Store for later use
results_activity1 = {
    'omega_1': omega_1,
    'omega_2': omega_2,
    'f_1': f_1,
    'f_2': f_2,
    'm_eff_1': m_eff_1,
    'm_eff_2': m_eff_2,
    'k_1': k_equiv_1,
    'k_2': omega_2**2 * m_eff_2
}



TWO-DEGREE-OF-FREEDOM MODEL SUMMARY



NameError: name 'omega_1' is not defined